In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

# 1️⃣ 读取文件
exposure = pd.read_csv("/data2/kunpeng/my_script/second_rotation/gwas_tool/filtered.tsv", sep="\t")
outcome = pd.read_csv("/data2/kunpeng/my_script/second_rotation/gwas_tool/filtered-7.tsv", sep="\t")

# 2️⃣ 按 SNP 合并
# 假设两者都有列 'SNP', 'BETA', 'SE'
df = pd.merge(exposure, outcome, on='SNP', suffixes=('.exp', '.out'))

# 3️⃣ 构造权重
df['weight'] = 1 / (df['SE.out'] ** 2)

# 4️⃣ IVW 回归（截距固定为0）
X_ivw = df['BETA.exp'].values.reshape(-1,1)
y_ivw = df['BETA.out'].values
w_ivw = df['weight'].values

ivw_model = sm.WLS(y_ivw, X_ivw, weights=w_ivw)
ivw_res = ivw_model.fit()
print("=== IVW regression (intercept=0) ===")
print(ivw_res.summary())

# 5️⃣ MR-Egger 回归（含截距）
X_egger = sm.add_constant(df['BETA.exp'].values)
egger_model = sm.WLS(y_ivw, X_egger, weights=w_ivw)
egger_res = egger_model.fit()
print("\n=== MR Egger regression ===")
print(egger_res.summary())

# 6️⃣ 提取结果
ivw_slope = ivw_res.params[0]
egger_intercept = egger_res.params[0]
egger_slope = egger_res.params[1]

print(f"\nIVW Slope = {ivw_slope}")
print(f"MR Egger Intercept = {egger_intercept}, Slope = {egger_slope}")


=== IVW regression (intercept=0) ===
                                 WLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.243
Model:                            WLS   Adj. R-squared (uncentered):              0.236
Method:                 Least Squares   F-statistic:                              34.06
Date:                Mon, 27 Oct 2025   Prob (F-statistic):                    5.89e-08
Time:                        21:24:59   Log-Likelihood:                          277.03
No. Observations:                 107   AIC:                                     -552.1
Df Residuals:                     106   BIC:                                     -549.4
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
----